##### Copyright 2023 The TF-Agents Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Agente REINFORCE

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/agents/tutorials/6_reinforce_tutorial"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/agents/tutorials/6_reinforce_tutorial.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/agents/tutorials/6_reinforce_tutorial.ipynb">     <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">     Ver código fuente en GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/agents/tutorials/6_reinforce_tutorial.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar bloc de notas</a>
</td>
</table>

## Introducción

En este ejemplo se muestra cómo usar la biblioteca de TF-Agents para entrenar un agente [REINFORCE](https://www-anw.cs.umass.edu/~barto/courses/cs687/williams92simple.pdf) en el entorno Cartpole. Esto es similar al [tutorial de DQN](1_dqn_tutorial.ipynb).

![Entorno Cartpole](images/cartpole.png)

Le ayudaremos a familiarizarse con todos los componentes de un proceso de Aprendizaje por Refuerzo (RL) para el entrenamiento, la evaluación y la recopilación de datos.


## Preparación

Si todavía no ha instalado las siguientes dependencias, ejecute estos comandos:

In [ ]:
!sudo apt-get update
!sudo apt-get install -y xvfb ffmpeg freeglut3-dev
!pip install 'imageio==2.4.0'
!pip install pyvirtualdisplay
!pip install tf-agents[reverb]
!pip install pyglet xvfbwrapper


In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import base64
import imageio
import IPython
import matplotlib.pyplot as plt
import numpy as np
import PIL.Image
import pyvirtualdisplay
import reverb

import tensorflow as tf

from tf_agents.agents.reinforce import reinforce_agent
from tf_agents.drivers import py_driver
from tf_agents.environments import suite_gym
from tf_agents.environments import tf_py_environment
from tf_agents.networks import actor_distribution_network
from tf_agents.policies import py_tf_eager_policy
from tf_agents.replay_buffers import reverb_replay_buffer
from tf_agents.replay_buffers import reverb_utils
from tf_agents.specs import tensor_spec
from tf_agents.trajectories import trajectory
from tf_agents.utils import common

# Set up a virtual display for rendering OpenAI gym environments.
display = pyvirtualdisplay.Display(visible=0, size=(1400, 900)).start()

## Hiperparámetros

In [ ]:
env_name = "CartPole-v0" # @param {type:"string"}
num_iterations = 250 # @param {type:"integer"}
collect_episodes_per_iteration = 2 # @param {type:"integer"}
replay_buffer_capacity = 2000 # @param {type:"integer"}

fc_layer_params = (100,)

learning_rate = 1e-3 # @param {type:"number"}
log_interval = 25 # @param {type:"integer"}
num_eval_episodes = 10 # @param {type:"integer"}
eval_interval = 50 # @param {type:"integer"}

## Entorno

Los entornos en RL representan la tarea o problema que tratamos de resolver. Los entornos estándar se pueden crear fácilmente en TF-Agents utilizando `suites`. Hay diferentes `suites` para cargar entornos desde fuentes como OpenAI Gym, Atari, DM Control, etc., dado un nombre de entorno de cadena.

Ahora, carguemos el entorno CartPole desde el paquete de OpenAI Gym.

In [ ]:
env = suite_gym.load(env_name)

Podemos renderizar este entorno para ver cómo luce. Un poste que oscila libremente está unido a un carro. El objetivo es mover el carro a la derecha o a la izquierda para lograr que el poste siga apuntando hacia arriba.

In [ ]:
#@test {"skip": true}
env.reset()
PIL.Image.fromarray(env.render())

La instrucción `time_step = environment.step(action)` ejecuta una `action` en el entorno.  La tupla `TimeStep` que devuelve contiene la siguiente observación del entorno y una recompensa para esa acción. Los métodos `time_step_spec()` y `action_spec()` en el entorno devuelven las especificaciones (tipos, formas, límites) de `time_step` y `action` respectivamente.

In [ ]:
print('Observation Spec:')
print(env.time_step_spec().observation)
print('Action Spec:')
print(env.action_spec())

Entonces, vemos que la observación es un arreglo de 4 flotantes: la posición y velocidad del carro, y la posición angular y velocidad del poste. Dado que solo se pueden realizar dos acciones (mover a la izquierda o mover a la derecha), la `action_spec` es un escalar donde 0 significa "mover a la izquierda" y 1 significa "mover a la derecha".

In [ ]:
time_step = env.reset()
print('Time step:')
print(time_step)

action = np.array(1, dtype=np.int32)

next_time_step = env.step(action)
print('Next time step:')
print(next_time_step)

Normalmente creamos dos entornos: uno para el entrenamiento y otro para la evaluación. La mayoría de los entornos están escritos en python puro, pero se pueden convertir fácilmente a TensorFlow con la ayuda del envoltorio `TFPyEnvironment`. La API del entorno original utiliza arreglos numpy, el `TFPyEnvironment` los convierte a/desde `Tensors` para que sea más fácil interactuar con las políticas y agentes de TensorFlow.


In [ ]:
train_py_env = suite_gym.load(env_name)
eval_py_env = suite_gym.load(env_name)

train_env = tf_py_environment.TFPyEnvironment(train_py_env)
eval_env = tf_py_environment.TFPyEnvironment(eval_py_env)

## Agente

El algoritmo que usamos para resolver un problema de RL se representa por un `Agent`. Además del agente REINFORCE, TF-Agents ofrece implementaciones estándar de una variedad de `Agents` como [DQN](https://storage.googleapis.com/deepmind-media/dqn/DQNNaturePaper.pdf), [DDPG](https://arxiv.org/pdf/1509.02971.pdf), [TD3](https://arxiv.org/pdf/1802.09477.pdf), [PPO](https://arxiv.org/abs/1707.06347) y [SAC](https://arxiv.org/abs/1801.01290).

Para crear un agente REINFORCE, lo primero que necesitamos es una `Actor Network` que pueda aprender a predecir la acción a partir de una observación del entorno.

Podemos crear fácilmente una `Actor Network` con ayuda de las especificaciones de las observaciones y las acciones. Podemos especificar las capas de la red que, en este ejemplo, son el argumento `fc_layer_params` establecido en una tupla de `ints` que representa los tamaños de cada capa oculta (Consulte la sección Hiperparámetros que figura más arriba).


In [ ]:
actor_net = actor_distribution_network.ActorDistributionNetwork(
    train_env.observation_spec(),
    train_env.action_spec(),
    fc_layer_params=fc_layer_params)

También necesitamos un `optimizer` para entrenar la red que acabamos de crear y una variable `train_step_counter` para hacer un seguimiento de la cantidad de veces que se actualizó la red.


In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

train_step_counter = tf.Variable(0)

tf_agent = reinforce_agent.ReinforceAgent(
    train_env.time_step_spec(),
    train_env.action_spec(),
    actor_network=actor_net,
    optimizer=optimizer,
    normalize_returns=True,
    train_step_counter=train_step_counter)
tf_agent.initialize()

## Políticas

En TF-Agents, las políticas representan la noción estándar de políticas en RL: dado que un `time_step` produce una acción o una distribución de acciones. El método principal es `policy_step = policy.action(time_step)` donde `policy_step` es una tupla nombrada `PolicyStep(action, state, info)`. `policy_step.action` es la `action` que se aplicará al entorno, `state` representa el estado de las políticas de estado (RNN), mientras que `info` podría contener información auxiliar como probabilidades logarítmicas de las acciones.

Los agentes contienen dos políticas: la política principal que se usa para la evaluación o la implementación (agent.policy) y otra política que se usa para la recopilación de datos (agent.collect_policy).

In [ ]:
eval_policy = tf_agent.policy
collect_policy = tf_agent.collect_policy

## Métricas y evaluación

La métrica más común que se usa para evaluar una política es el rendimiento medio. El rendimiento es la suma de las recompensas obtenidas al ejecutar una política en un entorno durante algunos episodios.  Podemos calcular la métrica de rendimiento medio de la siguiente manera.


In [ ]:
#@test {"skip": true}
def compute_avg_return(environment, policy, num_episodes=10):

  total_return = 0.0
  for _ in range(num_episodes):

    time_step = environment.reset()
    episode_return = 0.0

    while not time_step.is_last():
      action_step = policy.action(time_step)
      time_step = environment.step(action_step.action)
      episode_return += time_step.reward
    total_return += episode_return

  avg_return = total_return / num_episodes
  return avg_return.numpy()[0]


# Please also see the metrics module for standard implementations of different
# metrics.

## Búfer de repetición

Para hacer un seguimiento de los datos recogidos del entorno, se usará [Reverb](https://deepmind.com/research/open-source/Reverb), un sistema de repetición eficiente, extensible y fácil de usar de Deepmind. Este sistema almacena los datos de experiencia cuando recogemos trayectorias y se consume durante el entrenamiento.

Este búfer de repetición se construye a partir de especificaciones que describen los tensores que se van a almacenar, y que se pueden obtener del agente si se usa `tf_agent.collect_data_spec`.

In [ ]:
table_name = 'uniform_table'
replay_buffer_signature = tensor_spec.from_spec(
      tf_agent.collect_data_spec)
replay_buffer_signature = tensor_spec.add_outer_dim(
      replay_buffer_signature)
table = reverb.Table(
    table_name,
    max_size=replay_buffer_capacity,
    sampler=reverb.selectors.Uniform(),
    remover=reverb.selectors.Fifo(),
    rate_limiter=reverb.rate_limiters.MinSize(1),
    signature=replay_buffer_signature)

reverb_server = reverb.Server([table])

replay_buffer = reverb_replay_buffer.ReverbReplayBuffer(
    tf_agent.collect_data_spec,
    table_name=table_name,
    sequence_length=None,
    local_server=reverb_server)

rb_observer = reverb_utils.ReverbAddEpisodeObserver(
    replay_buffer.py_client,
    table_name,
    replay_buffer_capacity
)

Para la mayoría de los agentes, `collect_data_spec` es una tupla nombrada `Trajectory` que contiene la observación, la acción, la recompensa, etc.

## Recopilación de datos

Como REINFORCE aprende de episodios completos, definimos una función para recopilar un episodio mediante la política de recopilación de datos dada y guardar los datos (observaciones, acciones, recompensas, etc.) como trayectorias en el búfer de repetición. Aquí se utiliza 'PyDriver' para ejecutar el bucle de recopilación de experiencias. Puede obtener más información sobre el controlador TF Agents en nuestro [tutorial sobre controladores](https://www.tensorflow.org/agents/tutorials/4_drivers_tutorial).

In [ ]:
#@test {"skip": true}

def collect_episode(environment, policy, num_episodes):

  driver = py_driver.PyDriver(
    environment,
    py_tf_eager_policy.PyTFEagerPolicy(
      policy, use_tf_function=True),
    [rb_observer],
    max_episodes=num_episodes)
  initial_time_step = environment.reset()
  driver.run(initial_time_step)

## Entrenamiento del agente

El bucle de entrenamiento implica tanto la recopilación de datos del entorno como la optimización de las redes del agente. A lo largo del proceso, evaluaremos de vez en cuando la política del agente para ver nuestro rendimiento.

Lo que sigue tardará unos ~3 minutos en ejecutarse.

In [ ]:
#@test {"skip": true}
try:
  %%time
except:
  pass

# (Optional) Optimize by wrapping some of the code in a graph using TF function.
tf_agent.train = common.function(tf_agent.train)

# Reset the train step
tf_agent.train_step_counter.assign(0)

# Evaluate the agent's policy once before training.
avg_return = compute_avg_return(eval_env, tf_agent.policy, num_eval_episodes)
returns = [avg_return]

for _ in range(num_iterations):

  # Collect a few episodes using collect_policy and save to the replay buffer.
  collect_episode(
      train_py_env, tf_agent.collect_policy, collect_episodes_per_iteration)

  # Use data from the buffer and update the agent's network.
  iterator = iter(replay_buffer.as_dataset(sample_batch_size=1))
  trajectories, _ = next(iterator)
  train_loss = tf_agent.train(experience=trajectories)  

  replay_buffer.clear()

  step = tf_agent.train_step_counter.numpy()

  if step % log_interval == 0:
    print('step = {0}: loss = {1}'.format(step, train_loss.loss))

  if step % eval_interval == 0:
    avg_return = compute_avg_return(eval_env, tf_agent.policy, num_eval_episodes)
    print('step = {0}: Average Return = {1}'.format(step, avg_return))
    returns.append(avg_return)

## Visualización


### Gráficos

Podemos representar gráficamente una comparativa entre el rendimiento y los pasos globales para comprobar el rendimiento de nuestro agente. `Cartpole-v0`, el entorno da una recompensa de +1 por cada paso de tiempo que el poste se mantiene erguido, y como la cantidad máxima de pasos es 200, el máximo rendimiento posible también es 200.

In [ ]:
#@test {"skip": true}

steps = range(0, num_iterations + 1, eval_interval)
plt.plot(steps, returns)
plt.ylabel('Average Return')
plt.xlabel('Step')
plt.ylim(top=250)

### Videos

Resulta útil visualizar el funcionamiento de un agente mediante el renderizado del entorno en cada paso. Pero antes de hacerlo, creemos una función para insertar videos en este colab.

In [ ]:
def embed_mp4(filename):
  """Embeds an mp4 file in the notebook."""
  video = open(filename,'rb').read()
  b64 = base64.b64encode(video)
  tag = '''
  <video width="640" height="480" controls>
    <source src="data:video/mp4;base64,{0}" type="video/mp4">
  Your browser does not support the video tag.
  </video>'''.format(b64.decode())

  return IPython.display.HTML(tag)

El siguiente código visualiza la política del agente para algunos episodios:

In [ ]:
num_episodes = 3
video_filename = 'imageio.mp4'
with imageio.get_writer(video_filename, fps=60) as video:
  for _ in range(num_episodes):
    time_step = eval_env.reset()
    video.append_data(eval_py_env.render())
    while not time_step.is_last():
      action_step = tf_agent.policy.action(time_step)
      time_step = eval_env.step(action_step.action)
      video.append_data(eval_py_env.render())

embed_mp4(video_filename)